In [ ]:
import os
from datetime import datetime

import numpy as np

import tensorflow as tf
print(tf.__version__)
from tensorflow import keras
os.environ["OMP_NUM_THREADS"] = "2"
tf.config.threading.set_intra_op_parallelism_threads(1)
tf.config.threading.set_inter_op_parallelism_threads(1)

DAY_SECONDS = 24 * 60 * 60

from music_nextsim_tuning import read_and_scale,get_good_columns1, train_autoenc_predict, plot_rmse, plot_all_rmse

In [ ]:
idir = '../../music_matrix/cfg01_m20'
max_date = datetime(2007, 4, 12)
max_std = 1.5
rmse_std_factor = 2.5
epochs = 20

In [ ]:
param_names, inp_rgps, inp_ftrs = read_and_scale(idir, max_date)
good_columns1 = get_good_columns1(inp_rgps, max_std)
print(idir.split('/')[-1], "AVG/STD filtering", len(good_columns1), good_columns1)
tf0, tp0, rf0, rp0, hi = train_autoenc_predict(inp_ftrs, inp_rgps, inp_ftrs.columns, epochs, keras)
rmse_n0, rmse_r0 = plot_rmse(tf0, tp0, rf0, rp0, hi)
tf1, tp1, rf1, rp1, hi = train_autoenc_predict(inp_ftrs, inp_rgps, good_columns1, epochs, keras)
rmse_n1, rmse_r1 = plot_rmse(tf1, tp1, rf1, rp1, hi)
# features filtered only by RMSE
good_columns2 = inp_ftrs.columns[(rmse_n0.to_numpy() < rmse_n0.std() * rmse_std_factor)]
print(idir.split('/')[-1], "RMSE filtering", len(good_columns2), good_columns2)
tf2, tp2, rf2, rp2, hi = train_autoenc_predict(inp_ftrs, inp_rgps, good_columns2, epochs, keras)
rmse_n2, rmse_r2 = plot_rmse(tf2, tp2, rf2, rp2, hi)
# features filtered by AVG/STD first and then by RMSE
good_columns3 = good_columns1[(rmse_n1.to_numpy() < rmse_n0.std()*rmse_std_factor)]
print(idir.split('/')[-1], "AVG/STD + RMSE filtering", len(good_columns3), good_columns3)
# features filtered by AVG/STD first and then by RMSE
good_columns3 = good_columns1[(rmse_n1.to_numpy() < rmse_n0.std()*rmse_std_factor)]
print(idir.split('/')[-1], "AVG/STD + RMSE filtering", len(good_columns3), good_columns3)
rmse_r_list = [rmse_n0, rmse_n1, rmse_n2]
rmse_n_list = [rmse_r0, rmse_r1, rmse_r2]
tf_list = [tf0, tf1, tf2]
tp_list = [tp0, tp1, tp2]
rf_list = [rf0, rf1, rf2]
rp_list = [rp0, rp1, rp2]
plot_all_rmse(rmse_r_list, rmse_n_list, tf_list, tp_list, rf_list, rp_list)
np.savez(f'tuning_paper_figures/filter_autoencoder.npz',
            rmse_n0=rmse_n0, rmse_r0=rmse_r0,
            rmse_n1=rmse_n1, rmse_r1=rmse_r1, good_columns1=good_columns1,
            rmse_n2=rmse_n2, rmse_r2=rmse_r2, good_columns2=good_columns2,
        )
